<a href="https://colab.research.google.com/github/elpiprin/NLP-Stuff/blob/main/RNN_Custom.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Setup**

***Imports***

In [1]:
#Source: https://www.tensorflow.org/text/tutorials/text_generation
import tensorflow as tf
import numpy as np
import os
import time

***Reading the data***

In [2]:
#Read, then decode for py2 compat.
text = open('/content/Articles.txt', 'rb').read().decode(encoding='utf-8')
#length of text is the number of characters in it
print(f'Length of text: {len(text)} characters')

Length of text: 480143 characters


In [3]:
#Take a look at the first 250 characters in text
print(text[:250])

Melissa Caddick: Death of Australian fraudster remains a mystery

When conwoman Melissa Caddick vanished from her luxurious eastern Sydney home in November 2020 - with only her partially decomposed foot found washed up on a beach months later - it 


In [4]:
#The unique characters in the file
vocab = sorted(set(text))
print(f'{len(vocab)} unique characters')

105 unique characters


### **Processing the text**

***Vectorising the text***

In [5]:
example_texts = ['abcdefg', 'xyz']

chars = tf.strings.unicode_split(example_texts, input_encoding='UTF-8')
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [6]:
ids_from_chars = tf.keras.layers.StringLookup(
    vocabulary=list(vocab), mask_token=None)

In [7]:
ids = ids_from_chars(chars)
ids

<tf.RaggedTensor [[59, 60, 61, 62, 63, 64, 65], [82, 83, 84]]>

In [8]:
chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)

In [9]:
chars = chars_from_ids(ids)
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [10]:
tf.strings.reduce_join(chars, axis=-1).numpy()

array([b'abcdefg', b'xyz'], dtype=object)

In [11]:
def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

***Prediction task*** 

***Creating training examples and targets*** 

In [12]:
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids

<tf.Tensor: shape=(480143,), dtype=int64, numpy=array([43, 63, 70, ..., 76, 77, 16])>

In [13]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [14]:
for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode('utf-8'))

M
e
l
i
s
s
a
 
C
a


In [15]:
seq_length = 100

In [16]:
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

for seq in sequences.take(1):
  print(chars_from_ids(seq))

tf.Tensor(
[b'M' b'e' b'l' b'i' b's' b's' b'a' b' ' b'C' b'a' b'd' b'd' b'i' b'c'
 b'k' b':' b' ' b'D' b'e' b'a' b't' b'h' b' ' b'o' b'f' b' ' b'A' b'u'
 b's' b't' b'r' b'a' b'l' b'i' b'a' b'n' b' ' b'f' b'r' b'a' b'u' b'd'
 b's' b't' b'e' b'r' b' ' b'r' b'e' b'm' b'a' b'i' b'n' b's' b' ' b'a'
 b' ' b'm' b'y' b's' b't' b'e' b'r' b'y' b'\r' b'\n' b'\r' b'\n' b'W' b'h'
 b'e' b'n' b' ' b'c' b'o' b'n' b'w' b'o' b'm' b'a' b'n' b' ' b'M' b'e'
 b'l' b'i' b's' b's' b'a' b' ' b'C' b'a' b'd' b'd' b'i' b'c' b'k' b' '
 b'v' b'a' b'n'], shape=(101,), dtype=string)


In [17]:
for seq in sequences.take(5):
  print(text_from_ids(seq).numpy())

b'Melissa Caddick: Death of Australian fraudster remains a mystery\r\n\r\nWhen conwoman Melissa Caddick van'
b'ished from her luxurious eastern Sydney home in November 2020 - with only her partially decomposed fo'
b'ot found washed up on a beach months later - it set off a frenzy in Australia.\r\n\r\nThe case blindsided'
b' investors, baffled police, and captured the imagination of a nation.\r\n\r\nThe fraudster has inspired a'
b' hit podcast, a TV dramatisation, and countless outlandish theories - including that she had been swa'


In [18]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [19]:
split_input_target(list("Tensorflow"))

(['T', 'e', 'n', 's', 'o', 'r', 'f', 'l', 'o'],
 ['e', 'n', 's', 'o', 'r', 'f', 'l', 'o', 'w'])

In [20]:
dataset = sequences.map(split_input_target)

In [21]:
for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : b'Melissa Caddick: Death of Australian fraudster remains a mystery\r\n\r\nWhen conwoman Melissa Caddick va'
Target: b'elissa Caddick: Death of Australian fraudster remains a mystery\r\n\r\nWhen conwoman Melissa Caddick van'


***Creating training batches***

In [22]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<_PrefetchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

### **Building the model**

In [23]:
# Length of the vocabulary in StringLookup Layer
vocab_size = len(ids_from_chars.get_vocabulary())

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [24]:
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

In [25]:
model = MyModel(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

### **Trying the model**

In [26]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 106) # (batch_size, sequence_length, vocab_size)


In [27]:
model.summary()

Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  27136     
                                                                 
 gru (GRU)                   multiple                  3938304   
                                                                 
 dense (Dense)               multiple                  108650    
                                                                 
Total params: 4,074,090
Trainable params: 4,074,090
Non-trainable params: 0
_________________________________________________________________


In [28]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

In [29]:
sampled_indices

array([103,  81,  32,  52,  40,  86, 105,  49,   2,  28,  59,  50,  89,
        59,   7,  73, 103,   0,  49,  82,  11,  52,  18,  42,   4,  20,
        25,  79,  33,  23,  28,  84,  36,  74,  26,  93,  99,  84,  66,
        28,  58,  41,  91,  75,  40,  51,  59,  11,  97,  84,  23,  70,
         6,  95,  37,  44,  83,  50,  77,  30, 102,  84,  75,  55,  88,
        12,  14,   9,  76,  11,  44,  49,  82,  88,  93,  87,  61,  15,
        98,   9,  73,  24,  92,  26,  76,   8,  24,  91,  63,  25, 102,
        99,  30,  56,  42,  71,  66,  98,   2,  79])

In [30]:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

Input:
 b'Under reforms made to Qatar\xe2\x80\x99s labour laws in 2020, employers and workers can end contracts early, bu'

Next Char Predictions:
 b'\xe2\x80\xa6wBVJ\xc3\xa1\xe2\x82\xacS\r:aT\xc3\xa9a$o\xe2\x80\xa6[UNK]Sx(V0L!27uC5:zFp8\xc3\xb6\xe2\x80\x98zh:]K\xc3\xb1qJUa(\xe2\x80\x93z5l#\xc3\xbcGNyTs?\xe2\x80\x9dzqY\xc3\xa8),&r(NSx\xc3\xa8\xc3\xb6\xc3\xa3c-\xe2\x80\x94&o6\xc3\xb38r%6\xc3\xb1e7\xe2\x80\x9d\xe2\x80\x98?ZLmh\xe2\x80\x94\ru'


### **Training the model**

***Attaching an optimiser & a loss function***

In [31]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [32]:
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", example_batch_mean_loss)

Prediction shape:  (64, 100, 106)  # (batch_size, sequence_length, vocab_size)
Mean loss:         tf.Tensor(4.6626196, shape=(), dtype=float32)


In [33]:
tf.exp(example_batch_mean_loss).numpy()

105.91317

In [34]:
model.compile(optimizer='adam', loss=loss)

***Configuring checkpoints***

In [35]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

***Executing the training***

In [36]:
EPOCHS = 60

In [37]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/60
74/74 [==============================] - 410s 5s/step - loss: 3.1432
Epoch 2/60
74/74 [==============================] - 407s 5s/step - loss: 2.3965
Epoch 3/60
74/74 [==============================] - 407s 5s/step - loss: 2.2080
Epoch 4/60
74/74 [==============================] - 412s 6s/step - loss: 2.0481
Epoch 5/60
74/74 [==============================] - 406s 5s/step - loss: 1.9003
Epoch 6/60
74/74 [==============================] - 411s 6s/step - loss: 1.7557
Epoch 7/60
74/74 [==============================] - 406s 5s/step - loss: 1.6282
Epoch 8/60
74/74 [==============================] - 410s 6s/step - loss: 1.5228
Epoch 9/60
74/74 [==============================] - 404s 5s/step - loss: 1.4362
Epoch 10/60
74/74 [==============================] - 401s 5s/step - loss: 1.3637
Epoch 11/60
74/74 [==============================] - 408s 6s/step - loss: 1.3008
Epoch 12/60
74/74 [==============================] - 403s 5s/step - loss: 1.2438
Epoch 13/60
74/74 [==================

### **Generating Text**

In [41]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=0.5):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [42]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

In [43]:
#Text 1
start = time.time()
states = None
next_char = tf.constant(['Activists'])
result = [next_char]

for n in range(11500):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

Activists have long said they want to see abortion protection, and preventing illness, a long-sought goal of the pandemic.

As pro-democracy provided live many factors for the army - they were not trained in secularism."

Mr Pradhan says that the services are doing the workers had been deported, but said: “Qatar does not arrest or depticular contest on a wave of public solidarity. Britain came second, which is why it was selected to run for the presidency.

True to his background, he has also maintained strict financial complaints committee (ICC) for sexual harassment since 2018 and quit recently, turkets reported mental broadcast in time in the country - they need to be able to conduct its own offensive operations, and exhausted its army and supplies.

But it's not just air support for ground troopers, yet the process of both sides appear to be progressive. They said it was in the office and he told me that I was hormonal," she said.

"It was like Russian Roulette. Either put a grieva

In [44]:
#Text 2
start = time.time()
states = None
next_char = tf.constant(['President'])
result = [next_char]

for n in range(11500):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

President on Thursday calling on him to invoke the 14th Amendment.

Senate Majority Whip Dick Durber Sarea Sinch Remonds - a decade of a statement, it said: "Since the 737 Max returned to service, airlines have flown nearly 240,000 flights explain.

“That relatives and some admirers of the state's Provinion gassep catastropher Sha Dissinger Institute (RAS, and their dependents, could lead to hij account of what had happened, Magistrate Ryan said on Thursday.

Investing out the correction in the bond market may be almost completely understudied given the amount of suffering it has caused.

When the brianes of leaders are ungeoponance in which the attorney general has ever been impeached and removed from office. The only two elected officials to take the leap.

"With all the prices going up this year, we thought this would be ideally been in the forefront of all the main battles in check. But the panic was only temporary – his guide righted the boat, and experts believe that long Covid i

In [45]:
#Text 3
start = time.time()
states = None
next_char = tf.constant(['Conflict'])
result = [next_char]

for n in range(11500):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

Conflicting court in the next six years at one percent growth. It also includes SNAP and TANF working at the many schools he attended as his family following his father) did to be in prison. So I had to tell him,” he says.

What’s been the most difficult ones for a flight to Boston, during which time the 4-year-old boy began complaining about the slow pace of the proceedings.

"The stakes associated with the game with the Senate, where Republicans have a 19-12 majority of Catholics.

The district offers bilingual seals on students’ transcripts signifying proficiency in two language used in the report can be seen as an attempt to channel the Great Patrick.

In the months leading up to the World Cup, States Gavein Kiestless to the Security Council in March, the head of the UN Assistance Mission in February 2023 for a live ben easy to treat two days after the massacre, he found a place that was "devastated… it looked haunt transifications during the US and Europe directly into the war.

A

In [48]:
#Text 4
start = time.time()
states = None
next_char = tf.constant(['Asylum'])
result = [next_char]

for n in range(11500):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

Asylum as a fundamental liberal right.

Both parties are struggling in the polls, making them even more shared - on WhatsApp past. Our relation to Ms Harper's experience, Police Scotland said: "The stories I'm hearing from people do seem to be exacerbated now it's one force.

"You have to be prepared to have a major infauroun of the attacks on Muslims.

"If you look at official data, there were 160 Hindus among the 200 people who won’t forget it are the families and it they were ready to higher starts in and was chased relentlessly by debt collectors.

"Financially it really wiped me away. I had to displace Joe Biden,” said Bob Howland, who was visiting the Kennedy library for the aced 1.5 million people in England, stretching from the 331st remembers the Great Patrick, said that the years - but police initially explored two theories - that Caddick was say.

In a statement, driving a web are better than the S&P 500. There’s an unfair advantage for [legal cases.

Although the BNO scheme

In [53]:
#Text 5
start = time.time()
states = None
next_char = tf.constant(['Corruption'])
result = [next_char]

for n in range(11500):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

Corruption crackdown is Xi’s signature campaign. It has swept across the ruling Communist Party, the great bubbles with overpriced marketing is one calpaign to reach Kyiv, the International Monetary Faves of the long run.

Some think that the deadlock is also a sign that Party and a five Kingly says. “It’s like drinking from a firehose. You’re absorbing so fast.”

As the West sime other gaph less search by the end you're convinced that Arnold was dead when local of about 10% for recently in Mayhill babs, a similar stronghold and Pakistan's most populous province. The telecommunication activists were danger of big sentences are struggling to raise the steel of air defense systems can also be used for offensive purposes.

But this time it may prove harder to prevent the baby and became the party for years to come.

For more than six decades, the reverence to the Kennedy family and a strong supporter of his message against government many cultural institutions, Kalakshetra, has been embro

In [54]:
#Text 6
start = time.time()
states = None
next_char = tf.constant(['Investigation'])
result = [next_char]

for n in range(11500):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

Investigation activists were handed a prison sentence which was not suspended.

Two men received sentences in the first round of testing where results were almost universally and will wing with cleing and took died against the concerns of business. Many of the complaints of the political economy and jobs through religious nationalism.

"The religious divide or the two headlises in his open from a bridge in the Spanish capital.

The agreement reached on Saturday phases in food stamp time lite at Gours and her feel away from companies like Microsoft, Mastercard, Chobani, Duoling, Colound top of LinkedIn, the security source said.

But for pro-choice Christians, the major African countries didn't think there would be any violence in our village. But on that day, PAC personnel had descended on Hashimat has fallen equipment, says Evolv has exaggerated how effective the system is.

"There's an unfair air clashes with who was sad that people want to move on from the pandemic, but the virus is

In [55]:
#Text 7
start = time.time()
states = None
next_char = tf.constant(['World'])
result = [next_char]

for n in range(11500):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

World Cup, but still including her safety regulations on social media have become accustomy access to often confidential informalization and throats who well of their darker complexion.

But the most alarming allegations were about abortion medication from outside her state - or the country - who is there to punch through than the razor wire and mines group with a live man who had uploaded it. “It said, ‘Hey, I’m trying to find out more infrivent after allegations against all three Gundecha brothers. Umakant and Akhilesh, have since been accused of sexual assault

Her district was so nearly three years ago, and contumination was for younger very managers a political power and mental well-being to technique. African in the service divisions accused of enforcing this ban through violence, often in the first ITV Modi's Bharaty earned his family, in Kampala before entering into politics in 2016 and joining the Forum for the ability to terrorize the civilian population of Ukraine using Jarr

In [56]:
#Text 8
start = time.time()
states = None
next_char = tf.constant(['Pandemic'])
result = [next_char]

for n in range(11500):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

Pandemic Zoom with former chairman of China Everbright Group — one of the country’s largest public hospital, Simão Mendes.

But Guinea-Bissau was in the grip of a national government was not followed for such serious allegations," he adds. "I can't imagine dost take of where the anti-government, they say, has stepped up their own abilities to real estate that caught at the same time as the group.

Germany last year said it had foiled an attempt by an economic downfall in principle. We still have a lot of work requirements for certain social safety net programs had remained stomach points. Its leader has since struggled to go beyond winning a were prediction. The should had her own on-air clashes with Schofield.)

Twitter as pick in military coups, and, on other occasions, pulling levers behind the scenes.

Many affects national leaders to post personal content.

Maybe the "Facebook for some accusations of heavy-handed policing. But a second big events for Hong Kong people, the entire s

In [68]:
#Text 9
start = time.time()
states = None
next_char = tf.constant(['Police'])
result = [next_char]

for n in range(11500):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

Police Scotland and any assertion otherwise is without foundation.

"Our resolute determination not to give up more energy. This means they become more intense and frequent natural disasters. That will disrupt the supply demanding what they and hundreds of their colleagues were need for what happened at all. A lot of people didn't one year or two," Matthew says.

While he admitted the hard seven has a computer system that automatically helps us to detect the target, capture it and they will strike it.

"So, you would have to do lots of them. Then you're into the ammunitities after he was reportedly detained by authorities as part of a government investigation. Anbang at the moment is more interesting."

She believes that while LinkedIn has evolved from leaving, they say it’s the same thing that happened here in GDR times and we don’t know if we will come home alive or not,” he added.

Despite the Taliban are fighting the group — which the US estimates that about whether these kinds of 

In [65]:
#Text 10
start = time.time()
states = None
next_char = tf.constant(['Treason'])
result = [next_char]

for n in range(11500):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

Treason and particularly in high-value areas like seeds.

Yet in friends, the information sent to Facebook during a test by the Obsarvaig and the four weapons.

“When I first met him, I thought that he was not a revolutionary leader but an evolutionary from the countryside to Garissa town, which sits on the Tanside to the UK by taught and by the people of Pakistanis - from delivery drivers to the tech community - rely on the Internet to earn a limit firearm.

The final straw was an email in 2018 to pull off. He adds that they have released a copy of the controversy in the Western Hemisphere.

But Harris’ staff and allies counter that the group would be able to conduct its own offensive operations, and exhausted its army and supplies.

But in the service he has spoken to the White House about the potential use of the 14th Amendment, I think it’s a legitimate constitutional question, but inserting it into the sea to determine how shark behaviour or ocean currents could have in this war.


### **Exporting the generator**

In [66]:
tf.saved_model.save(one_step_model, 'one_step')
one_step_reloaded = tf.saved_model.load('one_step')

In [67]:
states = None
next_char = tf.constant(['Trial'])
result = [next_char]

for n in range(100):
  next_char, states = one_step_reloaded.generate_one_step(next_char, states=states)
  result.append(next_char)

print(tf.strings.join(result)[0].numpy().decode("utf-8"))

Trial called the finding to Britton.

Westover said he was finally convinced that Arnold was dead when 
